In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv("../data/gpt_generated_data/all_12_with_context2_train.csv")
df2 = pd.read_csv("../data/gpt_generated_data/science_qa.csv")
df3 = pd.read_csv("../data/gpt_generated_data/openbook.csv")
df4 = pd.read_csv("../data/gpt_generated_data/mmlu.csv")
# df6 = pd.read_csv("../data/gpt_generated_data/20230914125028_20230911_gpt3.5_generate4.csv")

In [24]:
df1["source"].value_counts()

3     14831
10    11679
2      5986
6      5968
5      5923
4      5862
8      3249
9      3069
7       999
11      993
1       500
Name: source, dtype: int64

In [4]:
import numpy as np
def fill_random(df):
    all_choices = df[["A", "B", "C", "D", "E"]].values.flatten()
    all_choices = all_choices[all_choices == all_choices]  # not nan
    def fillna_randomly(value):
        if pd.isna(value):
            return np.random.choice(all_choices)
        else:
            return value

    return df.applymap(fillna_randomly)
df1 = fill_random(df1)
df2 = fill_random(df2)
df3 = fill_random(df3)
df4 = fill_random(df4)

In [5]:
len(df1), len(df2), len(df3), len(df4)

(59059, 3291, 2115, 10915)

In [8]:
df = pd.concat([df1, df2, df3, df4])[['prompt', 'A', 'B', 'C', 'D', 'E', 'answer', 'source']]

In [10]:
df["source"].value_counts()

3                                                                                  14831
10                                                                                 11679
2                                                                                   5986
6                                                                                   5968
5                                                                                   5923
4                                                                                   5862
additional_data/MMLU/test.csv                                                       5781
additional_data/MMLU/train.csv                                                      4499
8                                                                                   3249
9                                                                                   3069
additional_data/ScienceQA/train.parquet                                             1995
additional_data/OpenB

In [26]:
df_valid = pd.concat([
    df[df["source"] == 1].sample(300),
    df[df["source"] == 2].sample(300),
    df[df["source"] == 3].sample(300),
    df[df["source"] == 4].sample(300),
    df[df["source"] == 5].sample(300),
    df[df["source"] == 6].sample(300),
    df[df["source"] == 7].sample(300),
    df[df["source"] == 8].sample(300),
    df[df["source"] == 9].sample(300),
    df[df["source"] == 11].sample(300),
    df[df["source"] == "additional_data/MMLU/test.csv"].sample(300),
    df[df["source"] == "additional_data/OpenBookQA-V1-Sep2018/OpenBookQA-V1-Sep2018/Data/Main/train.tsv"].sample(300),
    df[df["source"] == "additional_data/ScienceQA/test.parquet"].sample(300),
])
df_valid["dataset"] = "valid"

In [27]:
df_train = df[~df.index.isin(df_valid.index)]
df_train["dataset"] = "train"

/tmp/ipykernel_42537/2436017171.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train["dataset"] = "train"


In [28]:
len(df), len(df_train), len(df_valid)

(75380, 68541, 3900)

In [29]:
pd.concat([df_train, df_valid]).groupby(["source", "dataset"]).size()

source                                                                           dataset
1                                                                                train        103
                                                                                 valid        300
2                                                                                train       5120
                                                                                 valid        300
3                                                                                train      14531
                                                                                 valid        300
4                                                                                train       5562
                                                                                 valid        300
5                                                                                train       5623
                             

In [30]:
pd.concat([df_train, df_valid]).reset_index(drop=True).to_csv("20230922_concatenate.csv")